# SHOWTIME

In [1]:
import pandas as pd
import os
import mlflow
import json

In [2]:
playlists_link = {
    "CoryKenshinRandomVideos": "https://www.youtube.com/playlist?list=PLxoHK1S7LhWSU230SAT7fDUsLiXCwaBMo",
    "MusicVideos": "https://youtube.com/playlist?list=PLrhpb4TQr-uKzxOB1C_9x-Ysrj2WRMZN_&si=SiaK1ztgFATZg10B",
    # "RandomVideos": "https://youtube.com/playlist?list=PLv3TTBr1W_9tppikBxAE_G6qjWdBljBHJ&si=tcnc1i_SwIo1wgdP", (3521 Videos, use with caution maybe AWOKWAOKAWOKAW)
}
project_directory = "/home/vane/dataScience/datastuff"
os.chdir(project_directory)

[youtube] zyXmsVwZqX4: Downloading tv client config
[youtube] zyXmsVwZqX4: Downloading tv player API JSON
[youtube] zyXmsVwZqX4: Downloading ios player API JSON
[youtube] zyXmsVwZqX4: Downloading m3u8 information
[info] zyXmsVwZqX4: Downloading 1 format(s): 398+251
[info] Downloading video thumbnail 41 ...
[info] Video Thumbnail 41 does not exist
[info] Downloading video thumbnail 40 ...
[info] Video Thumbnail 40 does not exist
[info] Downloading video thumbnail 39 ...
[info] Video Thumbnail 39 does not exist
[info] Downloading video thumbnail 38 ...
[info] Video Thumbnail 38 does not exist
[info] Downloading video thumbnail 37 ...
[info] Writing video thumbnail 37 to: 162.webp
[info] Writing video metadata as JSON to: 162.info.json
[ThumbnailsConvertor] Converting thumbnail "162.webp" to jpg
Deleting original file 162.webp (pass -k to keep)
[download] Downloading item 163 of 728
[youtube] Extracting URL: https://www.youtube.com/watch?v=04F4xlWSFh0
[youtube] 04F4xlWSFh0: Downloading we

In [3]:
def playlist_download(playlists: dict[str, str]) -> None:
    os.chdir(os.path.join(os.getcwd(), "data"))
    for playlist_title, playlist_link in playlists.items():
        os.mkdir(f"{playlist_title}Playlist")
        os.chdir(os.path.join(os.getcwd(), f"{playlist_title}Playlist"))
        os.system(f"yt-dlp --write-info-json --skip-download --convert-thumbnails jpg --write-thumbnail -o \"%(playlist_index)s.%(ext)s\" {playlist_link}")
        os.chdir("..")

[info] Writing video thumbnail 41 to: 166.webp
[info] Writing video metadata as JSON to: 166.info.json
[ThumbnailsConvertor] Converting thumbnail "166.webp" to jpg
Deleting original file 166.webp (pass -k to keep)
[download] Downloading item 167 of 728
[youtube] Extracting URL: https://www.youtube.com/watch?v=a2RA0vsZXf8
[youtube] a2RA0vsZXf8: Downloading webpage
[youtube] a2RA0vsZXf8: Downloading tv client config
[youtube] a2RA0vsZXf8: Downloading tv player API JSON
[youtube] a2RA0vsZXf8: Downloading ios player API JSON
[youtube] a2RA0vsZXf8: Downloading m3u8 information
[info] a2RA0vsZXf8: Downloading 1 format(s): 616+251
[info] Downloading video thumbnail 41 ...
[info] Writing video thumbnail 41 to: 167.webp
[info] Writing video metadata as JSON to: 167.info.json
[ThumbnailsConvertor] Converting thumbnail "167.webp" to jpg
Deleting original file 167.webp (pass -k to keep)
[download] Downloading item 168 of 728
[youtube] Extracting URL: https://www.youtube.com/watch?v=z61GFxVD8K4
[yo

In [4]:
# playlist_download(playlists_link)

In [64]:
numerical_feature_json_keys = ["comment_count", "view_count"]
text_feature_json_keys = ["title", "description", "channel"]
categories_feature_json_keys = ["tags", "categories"]
duration_string_feature_json_key = "duration_string"
playlist_json_file = "000.info.json"

def duration_string_to_seconds(duration_string: str) -> int:
    multiplier = 1
    total_seconds = 0
    for digit in duration_string.split(":")[::-1]:
        total_seconds += int(digit) * multiplier
        multiplier *= 60
    return total_seconds

def video_json_to_dictionary(file_path: str) -> dict:
    with open(file_path, 'r') as f:
        result = dict()
        data = json.load(f)
        for key in numerical_feature_json_keys:
            result.update({key: data.get(key, 0)})
        for key in text_feature_json_keys:
            result.update({key: data.get(key, "")})
        for key in categories_feature_json_keys:
            result.update({key: " ".join(data.get(key, ""))})
        result.update({"duration": duration_string_to_seconds(data[duration_string_feature_json_key])})
    return result

def get_title_from_playlist_json(file_path: str) -> str:
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data["title"]

def playlist_to_dataframes() -> dict[str, pd.DataFrame]:
    dataframes = dict()
    for root, dirs, files in os.walk(os.path.join(os.getcwd(), "data")):
        dirs[:] = [d for d in dirs if not d[0] == '.']
        if len(files) == 0: continue
        playlist_dataframe = pd.DataFrame()
        playlist_title = ""
        files.sort()
        for file in files:
            print(file)
            file_path = os.path.join(root, file)
            if file == playlist_json_file:
                playlist_title = get_title_from_playlist_json(file_path)
            elif file.endswith(".json"):
                print(video_json_to_dictionary(file_path))
                tmp_dataframe = pd.DataFrame(video_json_to_dictionary(file_path), index=[int(file.split(".")[0])])
                playlist_dataframe = pd.concat([playlist_dataframe, tmp_dataframe], ignore_index=False)
        playlist_dataframe = playlist_dataframe.sort_index()
        dataframes.update({playlist_title: playlist_dataframe})

    return dataframes
    return dataframes

SyntaxError: invalid syntax (2177666736.py, line 42)

In [65]:
shit = playlist_to_dataframes()

000.info.json
000.jpg
001.info.json
{'comment_count': 4300000, 'view_count': 8747214036, 'title': 'Luis Fonsi - Despacito ft. Daddy Yankee', 'description': '“Despacito” disponible ya en todas las plataformas digitales: https://UMLE.lnk.to/DOoUzFp \n“Imposible” disponible ya en todas las plataformas digitales: https://UMLE.lnk.to/IMPOSIBLEFp\n“Calypso” disponible ya en todas las plataformas digitales: https://UMLE.lnk.to/CLYPSFp\nEchame La Culpa disponible ya en todas las plataformas digitales: https://UMLE.lnk.to/ELCFp\n\n\nBest of Luis Fonsi / Lo mejor Luis Fonsi: https://goo.gl/KLWPSa \nSubscribe here: https://goo.gl/nkhcGc\nSigue a Luis Fonsi: \nOfficial Site: http://www.luisfonsi.com/ \nFacebook: https://www.facebook.com/luisfonsi/ \nTwitter: https://twitter.com/LuisFonsi \nInstagram: https://www.instagram.com/luisfonsi\n\n#LuisFonsi #Despacito #Imposible #Calypso #EchamelaCulpa #NadaEsImposible #NothingisImpossible #LF\n\n\nMusic video by Luis Fonsi performing Despacito. (C) 2017 

In [66]:
shit['Random Videos']

,comment_count,view_count,title,description,channel,tags,categories,duration
1,295000,18269454,Reacting to the Five Nights at Freddy's Movie ...,Trailer: https://www.youtube.com/watch?v=0VH9W...,CoryxKenshin,,Gaming,423
2,100000,3968804,30,this video has no purpose other than for me to...,CoryxKenshin,30 coryxkenshin cory kenshin birthday 2022 rem...,Gaming,475
3,29000,4555645,I Can't Hide This Any Longer,https://coryxkenshin.com GOGOGOGO BEFORE IT SE...,CoryxKenshin,i cant hide this any longer cant hide this any...,Gaming,937
4,130000,10211607,Deleting My YouTube Channel.,Promise broken. Thank you for everything.,CoryxKenshin,deleting my youtube channel coryxkenshin cory ...,Gaming,1159
5,94000,10395203,YouTube: Racism and Favoritism,Never in a million years did I think I would u...,CoryxKenshin,youtube favoritism and racism coryxkenshin cor...,Gaming,838
...,...,...,...,...,...,...,...,...
182,169,26047,A Quick Thank You Vlog,Twitter: https://twitter.com/coryxkenshin\nFac...,CoryxKenshin,coryxkenshin cory kenshin quick thank you vlog...,People & Blogs,200
183,31000,2632806,"My Last YouTube Video, Goodbye YouTube..",Twitter: https://twitter.com/coryxkenshin\n\n\...,CoryxKenshin,coryxkenshin cory kenshin my last youtube vide...,People & Blogs,185
184,155,17601,Marriage/Easter Dinner/2500 Subscribers/NAENAE,FOLLOW MY TWITTER : @CoryxKenshin\nFACEBOOK: F...,CoryxKenshin,Marriage (Marriage/union Type) Easter (Holiday...,People & Blogs,239
185,171,63770,Jamming At Guitar Center! [Drum],"When I saw this baby sitting there, I couldn't...",CoryxKenshin,Drums (Musical Instrument) Jamming (Compositio...,People & Blogs,73
